In [1]:
import parse_tfl_json
import google

In [5]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime 
where tfl_message = 'ok' and station_name like '%olche%'
limit 100
"""
df = pd.read_sql(sql, conn)

In [6]:
from process_journeys import get_london_icscodes, add_cycle_and_total_time, remove_journeys_not_arriving_clondon
from google import get_cycle_info
import parse_tfl_json
import json

In [7]:
# df = df.loc[8:9,:]

In [8]:
for r in df.iterrows():
    index = r[0]
    row = r[1]

    try:
        j1 = row["tfl_response"]
        j2 = json.loads(j1)
    except:
        print("parse problem")
        continue
    
    try:
        journeys = j2["journeys"]
    except:
        print("no journeys in json")
        continue
    
    
    for journey in journeys:
        legs = journey["legs"]
        journey["legs"] = parse_tfl_json.remove_non_national_rail(legs)
    

    
    # Keep only journeys with a national rail component
    journeys = [j for j in journeys if len(j["legs"]) > 0]
    
    
    
    # Keep only journeys that end in central London
    
    journeys = remove_journeys_not_arriving_clondon(journeys)

    
    if len(journeys) == 0:
        continue

    journeys = add_cycle_and_total_time(journeys, row["tfl_dest"])

    journeys = sorted(journeys, key = lambda x: x["total_time"])

    journey = journeys[0]
    legs = journey["legs"]

    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary

    df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
    df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)

    df.loc[index, "depart_lat"] = lat_lng["depart"]["lat"]
    df.loc[index, "depart_lng"] = lat_lng["depart"]["lng"]
    df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
    df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]

    df.loc[index, "cycle_minutes"] = journey["cycle_minutes"]
    df.loc[index, "cycle_miles"] = journey["cycle_miles"]
    df.loc[index, "total_journeytime"] = journey["cycle_minutes"] + travel_time_minutes

    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict"]]
df2 = df[cols]

In [9]:
df2

,nlc,icscode,station_name,tfl_message,tfl_dest,natrail_journey_summary,natrail_train_changes,final_arrival,natrail_journey_minutes,depart_lat,depart_lng,arrive_lat,arrive_lng,cycle_minutes,cycle_miles,total_journeytime
0,6853,1000603,Colchester Town,ok,SW1H9AJ,Colchester Town Rail Station → Osborne Street ...,3.0,Stratford (London) Rail Station,66.0,51.886532,0.904520,51.542234,-0.002819,34.275712,7.997666,100.275712
1,6853,1000603,Colchester Town,ok,SW1H9AJ,Colchester Town Rail Station → Osborne Street ...,3.0,Liverpool Street Station,69.0,51.886532,0.904520,51.518380,-0.081093,16.742398,3.906559,85.742398
2,6853,1000603,Colchester Town,ok,SW1H9AJ,Colchester Town Rail Station → Osborne Street ...,4.0,Liverpool Street Station,83.0,51.886532,0.904520,51.518380,-0.081093,16.742398,3.906559,99.742398
3,6853,1000603,Colchester Town,ok,E145HP,Colchester Town Rail Station → Osborne Street ...,3.0,Stratford (London) Rail Station,66.0,51.886532,0.904520,51.542234,-0.002819,14.902252,3.477192,80.902252
4,6853,1000603,Colchester Town,ok,E145HP,Colchester Town Rail Station → Stratford (Lond...,0.0,Stratford (London) Rail Station,59.0,51.886454,0.904763,51.541856,-0.003341,14.902252,3.477192,73.902252
5,6853,1000603,Colchester Town,ok,E145HP,Colchester Town Rail Station → Osborne Street ...,3.0,Stratford (London) Rail Station,66.0,51.886532,0.904520,51.542234,-0.002819,14.902252,3.477192,80.902252
6,6861,1000600,Colchester,ok,SW1H9AJ,Colchester Rail Station → Stratford (London) R...,0.0,Stratford (London) Rail Station,43.0,51.900712,0.892598,51.542234,-0.002819,34.275712,7.997666,77.275712
7,6861,1000600,Colchester,ok,SW1H9AJ,Colchester Rail Station → Stratford (London) R...,0.0,Stratford (London) Rail Station,43.0,51.900712,0.892598,51.542234,-0.002819,34.275712,7.997666,77.275712
8,6861,1000600,Colchester,ok,SW1H9AJ,Colchester Rail Station → Stratford (London) R...,0.0,Stratford (London) Rail Station,43.0,51.900712,0.892598,51.542234,-0.002819,34.275712,7.997666,77.275712
9,6861,1000600,Colchester,ok,E145HP,Colchester Rail Station → Stratford (London) R...,0.0,Stratford (London) Rail Station,43.0,51.900712,0.892598,51.542234,-0.002819,14.902252,3.477192,57.902252


In [11]:
df3 = df2.sort_values(["station_name", "tfl_dest", "total_journeytime"]).drop_duplicates(["station_name", "tfl_dest"])
df3

,nlc,icscode,station_name,tfl_message,tfl_dest,natrail_journey_summary,natrail_train_changes,final_arrival,natrail_journey_minutes,depart_lat,depart_lng,arrive_lat,arrive_lng,cycle_minutes,cycle_miles,total_journeytime
9,6861,1000600,Colchester,ok,E145HP,Colchester Rail Station → Stratford (London) R...,0.0,Stratford (London) Rail Station,43.0,51.900712,0.892598,51.542234,-0.002819,14.902252,3.477192,57.902252
6,6861,1000600,Colchester,ok,SW1H9AJ,Colchester Rail Station → Stratford (London) R...,0.0,Stratford (London) Rail Station,43.0,51.900712,0.892598,51.542234,-0.002819,34.275712,7.997666,77.275712
4,6853,1000603,Colchester Town,ok,E145HP,Colchester Town Rail Station → Stratford (Lond...,0.0,Stratford (London) Rail Station,59.0,51.886454,0.904763,51.541856,-0.003341,14.902252,3.477192,73.902252
1,6853,1000603,Colchester Town,ok,SW1H9AJ,Colchester Town Rail Station → Osborne Street ...,3.0,Liverpool Street Station,69.0,51.886532,0.904520,51.518380,-0.081093,16.742398,3.906559,85.742398


In [ ]:
# Want to update all the names to be cw specific 
df5 = df4.rename(columns={"depart_lat" : "lat", "depart_lng": "lng", "station_name": "name"})

In [ ]:
df5.to_csv("cw_to_map.csv", encoding="utf-8", index=False)

In [ ]:
# Now want to spread so that we have one row per station, then metrics for both cw and pw on that row
